In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# !ls /content/drive/My\ Drive/JnOTE/graficos/1esp_15hz_1hz_1/
# !ls /content/drive/My\ Drive/JnOTE/graficos/
!ls /content/drive/My\ Drive/JnOTE/graficos/oth_ith_4/

dep_writing_Total_25_oth0.07_ith0.5.csv
dep_writing_Total_25_oth0.09_ith0.5.csv
dep_writing_Total_25_oth0.11_ith0.5.csv
dep_writing_Total_25_oth0.13_ith0.5.csv
dep_writing_Total_25_oth0.15_ith0.5.csv
dep_writing_Total_25_oth0.17_ith0.5.csv


In [ ]:
for name in dir():
       if not name.startswith('_'):
           del globals()[name]

In [ ]:
nn = 60
mm = 5
method = "OTH"
# pat = "/content/drive/My Drive/JnOTE/graficos/"
pat = "/content/drive/My Drive/JnOTE/graficos/oth_ith_4/"
# pat = "/content/drive/My Drive/JnOTE/graficos/1esp_15hz_1hz_1/"
fileN = "dep_writing_Total_25_oth0.15_ith0.5.csv"


# Reset Variables

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import math
import statistics as sta
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.manifold import Isomap
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import VotingClassifier
from random import seed
from random import randint
import random

from sklearn.metrics import roc_auc_score
from sklearn import metrics       
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve           

from sklearn import metrics 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve                         # thresholds = all unique prediction probabilities in descending order

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import LocallyLinearEmbedding

import time

In [ ]:
def agument_Data(x, Eps, proportion):
  N=int(proportion*len(x))
  df_to_be_agumented=x
  rn=rng.normal(0, Eps, size=(1, x.shape[1]))
  #print('rn=',rn)

  for i in range(N):                            # create new N data. Where N are the required proportion.
    rd_row=random.randint(0,len(x)-1)           # randomly select one row to include noise.
    #print('i',i,'rd_row',rd_row)
    b=x.values[rd_row,0:x.shape[1]]+rn          # add Gaussian Noise to the selected observation.
    b=pd.DataFrame(data=b)                      # return format to Panda Dataframe
    b.columns = x.columns                       # re-name the new dataframe with the original names of the columns
    b.values[0,b.shape[1]-1]=int(x.values[rd_row,b.shape[1]-1]) # convert to integer the last value (this is the value used to classify)
    df_to_be_agumented = pd.concat([df_to_be_agumented,b])      # concat the augmente observation at the end of the array.
    df_to_be_agumented.reset_index(inplace=True, drop=True)     # reset the index
  return(df_to_be_agumented)


In [ ]:

path = pat
file= path + fileN
data = pd.read_csv(file, header = None)
data_name = []
for i in range(len(data.axes[1])):

  if i == len(data.axes[1])-1:
    data_name = data_name + ['Emotions']
  else:
    data_name = data_name + [str(i)]

data_n = data_name
data.columns = data_name
def make_dataframes_classes_same_length(x, Eps, prop):
  e0=x[(x['Emotions']==0)]
  e1=x[(x['Emotions']==1)]
  l_e0=len(e0)
  l_e1=len(e1)
  #print('e0',len(e0))
  #print('e1',len(e1))
  if l_e0>l_e1:
      prop=l_e0/l_e1-1
      e1=agument_Data(e1, Eps, prop)
  else:
      prop=e1/l_e0-1
      e0=agument_Data(e0,prop, Eps)
  df_to_be_agumented = pd.concat([e0,e1])                   # concat the augmente observation at the end of the array.
  df_to_be_agumented.reset_index(inplace=True, drop=True)   # reset the index
  return df_to_be_agumented 


In [ ]:
e0=data[(data['Emotions']==0)]

In [ ]:
####################################################################################
# suffle_df: retruns the dataframes with segments i and j of length k shuffled.
#
# Input:
#   X is the input matrix to be shuffled.
#   i is the starting rows of the first segment of rows to shuffle
#   j is the starting rows of the second segment of rows to shuffle
#   k is the number of rows to shuffle
#  Output:
#   the dataframes with segments i and j of length k shuffled.
####################################################################################
def suffle_df(X, i, j, k):
  #a=X.iloc[0:8].copy()
  a=X
  #print(a)
  b,c=a.iloc[0:i+k].copy(),a.iloc[j:j+k].copy()
  b.reset_index(inplace=True, drop=True)
  c.reset_index(inplace=True, drop=True)
  b.index=b.index + j  # modify the values of the index such that 'b' variable can be replaced.
  #print(b)
  #print(c)
  a.iloc[0:i+k], a.iloc[j:j+k]=c,b
  return(a)


In [ ]:
# #PCA
# def PComp(dataset,n):
#   fDum = dataset.columns
#   features = fDum[0:-1] 
#   x = dataset.loc[:,features].values
#   y = dataset.loc[:,["Emotions"]].values
#   pca = PCA(n_components=n)
#   prinComp = pca.fit_transform(x)
#   names = []
#   for i in range(n):
#       names = names + ["P Component" + str(i+1)]
#   prinDf = pd.DataFrame(data = prinComp, columns = names)
#   finalDf = pd.concat([prinDf,dataset[["Emotions"]]], axis = 1)
#   return finalDf,pca
# data,pca = PComp(data, 20)
# data
#LLE
def comp(dataset,datas, n,m):
  # fDum = dataset.columns
  # features = fDum[0:-1] 
  # print(features)
  # x = dataset.loc[:,features].values
  # y = dataset.loc[:,["Emotions"]].values
  if method == "LLE":

    embedding = LocallyLinearEmbedding(n_components=n,n_neighbors=m)
    X_trans = embedding.fit_transform(dataset)
    X_test = embedding.transform(datas)
  elif method == "PCA":
    
    pca = PCA(n_components=n)
    X_trans = pca.fit_transform(dataset)
    X_test = pca.transform(datas)
  elif method == "ICA":
    ica = FastICA(n_components = n, random_state = 0, max_iter = 250, tol = 0.1)
    X_trans = ica.fit_transform(dataset)
    X_test = ica.transform(datas)
  elif method == "ISO":
    iso = Isomap(n_components = n, n_neighbors = m)
    X_trans = iso.fit_transform(dataset)
    X_test = iso.transform(datas)
  else:
    quit()
  names = []
  for i in range(n):
    names = names + ["Component " + str(i+1)]
  prinDf = pd.DataFrame(data = X_trans, columns = names)
  prinDf2 = pd.DataFrame(data = X_test, columns = None)
  # finalDf = pd.concat([prinDf,dataset[["Emotions"]]], axis = 1)
  # return finalDf
  return prinDf, prinDf2
# data_new = comp(data, nn,mm)


In [ ]:
####################################################################################
####################################################################################
####################################################################################
def train_test_sets(x, i, k, Eps, pr,n,m):
  X_test2=x.values[i:k,col]
  y_test=x.values[i:k,nf]
  tmp=x.loc[k:len(x)]
  #print('Input tmp.shape[0]',tmp.shape[0])
  tmp = make_dataframes_classes_same_length(tmp, Eps, pr)
  # print(tmp2.shape)
  # print(X_test2.shape)
  
  # X_test = LLEcomp(X_test,n,7)
  #tmp = agument_Data(tmp, Eps, pr)
  #print('Output tmp.shape[0]', tmp.shape[0])
  X_train2=tmp.values[k:len(tmp),col]
  y_train=tmp.values[k:len(tmp),nf]
  # X_train2=x.values[k:len(x),col]
  # y_train=x.values[k:len(x),nf]
  # X_train2=tmp.values[k:len(tmp),range(0,n+1)]
  # y_train=tmp.values[k:len(tmp),n]
  # print(X_train2.shape)
  # print(X_test2.shape)
  if "oth" in fileN:
    X_train, X_test = X_train2, X_test2
  else:
    X_train, X_test = comp(X_train2,X_test2,nn,m)

  # print(X_train.shape)
  # print(y_train.shape)
  # print(X_test.shape)
  # print(y_test.shape)
  #print('X_train.shape[0]', X_train.shape[0])
  #print('y_train.shape[0]', y_train.shape[0])

  return X_train, X_test, y_train, y_test


In [ ]:
def normalize(dataset):
    dataNorm=(dataset-dataset.mean())/(dataset.std())
    dataNorm["Emotions"]=dataset["Emotions"]
    return dataNorm
data=normalize(data)
data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,Emotions
0,-0.347844,2.189228,0.463538,-1.801627,0.632033,-0.553422,-0.455269,-0.366360,-0.462043,0.536388,-0.706399,-2.195129,-1.222446,-2.212059,-0.226695,0.774891,0.055841,0.632462,-1.079671,-1.090790,-1.017814,-2.230147,-2.489456,-0.907112,0.748609,4.267456,1.083494,1.487158,-0.635287,0.161437,-0.497632,-0.343546,0.132867,0.120168,-0.153347,0.017513,0.533959,-0.59592,-0.454652,0.042539,-2.175388,0.194210,1.426720,1.897496,-0.024237,1.893892,-0.983550,-0.591281,-0.459416,0.637440,-2.203682,-1.335505,-0.433110,-0.232504,-1.137321,-0.398924,-0.808538,0.697608,0.280723,-1.106731,-0.722058,-0.137730,-0.627346,0.216590,1.442091,-0.263486,2.551502,-0.164256,1.957830,0.805461,-1.241912,-0.154762,0
1,-0.503110,0.280254,-1.347963,-0.058510,-0.566242,-0.124410,-0.556104,-0.290405,-0.321090,2.001509,0.461881,-0.304357,1.182715,1.225488,-0.868779,-1.433517,-0.401772,0.592280,-0.022377,-0.369429,0.821072,-0.483921,0.441827,-0.085772,-0.716672,-0.416869,0.959260,-0.695879,-0.303459,0.853909,1.068091,-1.034253,-0.731130,-0.430382,0.102911,-0.251604,-0.145263,-0.59592,-0.883968,-0.518293,-0.802016,-0.482656,-0.659171,1.148316,0.044111,-0.610708,0.792701,1.475319,0.410914,-0.799359,-1.495614,-0.093415,-0.130537,0.094612,0.367879,0.706725,0.380194,-0.317259,0.147259,0.029110,0.132494,-0.832425,-0.475283,-0.522627,-0.273986,-0.016274,0.227323,-1.157530,-1.188340,-1.287310,-0.806115,-0.198937,0
2,-0.037312,0.484960,0.963926,-0.462755,0.156687,-0.002571,-0.005037,-0.349454,-0.634015,1.081400,-1.588844,-0.050649,-1.896508,-2.091472,0.692687,1.220831,-1.023721,-0.123929,-0.764112,-1.799578,-0.505701,2.111437,-0.884590,-0.700450,-0.296839,-0.300457,0.859547,-1.455747,-1.174632,1.614772,-0.166877,0.315606,0.187450,-0.706769,-0.236035,-0.191137,-0.704568,-0.59592,0.206790,0.893521,-0.448994,-0.507871,-0.030570,0.640251,0.007614,-0.363689,-0.232789,-1.540276,-1.766800,0.982071,1.018257,0.461155,-0.008208,0.890140,0.728218,0.603371,0.094229,0.303877,-0.161572,-0.376768,-0.657849,0.198502,-0.172130,-0.108749,0.803189,0.472073,-0.475788,-0.799827,0.972928,1.705026,-0.336069,-0.500178,0
3,-0.192578,-0.145812,2.744152,1.611851,-1.012134,-0.474206,1.844317,-0.797957,-0.766887,0.477078,-1.054581,-0.915834,-0.869565,0.419974,1.759775,0.032289,-0.276572,2.652707,0.601547,-0.924537,-1.733231,0.042723,0.265512,-1.649626,0.172891,0.616791,0.084723,0.608873,-0.619905,-0.441269,-0.162459,0.655700,0.269832,-0.444227,0.992210,0.008807,-0.167134,1.66507,-0.498751,-1.260515,-0.885717,0.370143,-1.705422,1.353264,0.119203,0.415127,-0.687849,0.524627,-0.963591,-1.167562,0.953530,-0.296722,-0.563166,-1.309302,-0.336701,-1.716442,-1.459843,-1.041672,3.177562,-0.841310,-0.260745,2.400550,0.449316,-0.648443,-0.433712,-0.312118,-0.504842,0.704725,1.765555,1.243591,0.667798,0.917145,1
4,-0.347844,-1.565319,-0.585353,-0.644989,0.578867,-0.231275,-1.853165,-0.391142,-0.696378,1.020487,-0.422071,0.500134,-0.630630,-0.530759,0.134701,-0.306523,-0.299102,-0.653630,0.525456,0.155408,-1.387807,0.366157,0.811176,0.756806,-0.630264,-0.681392,0.689543,-0.360309,-0.534427,0.980719,-0.636051,-0.753763,0.253898,0.172919,0.150632,-0.035486,-0.230691,-0.59592,0.350717,0.748955,-0.175686,-0.591145,-0.642877,-0.498159,0.450074,-0.356621,0.000505,0.238329,-1.354130,-0.253930,0.216695,0.214909,-0.785781,-0.659238,-0.743665,-0.471888,-1.019763,0.165956,-0.740047,-0.289926,0.230568,-1.374269,1.322001,-0.125273,-1.225522,0.139754,-0.446733,-1.151674,1.669012,0.649319,-1.628106,0.197839,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,-0.037312,0.7

In [ ]:
# lst = []
# lst = [index for index,value in enumerate(pca.explained_variance_) if value > 1]
# data,pca = PComp(data, len(lst))
# data

# Training and Testing Function

In [ ]:
####################################################################################
####################################################################################
####################################################################################
start_time=time.time()
print(method)
print(nn)
print(mm)
print(fileN)
LR_max_iter=500

#Eps=0.1; prop=0.2; **w= 23 j= 1 average accuracy= 0.6640625
Eps=0.2 #por la normalización
prop=0.2

CV=5
k=1
acc_max=0
for w in range(200,400):
   for j in range(200,400):
     path = pat
     file= path + fileN

     data = pd.read_csv(file, header = None)
     data_name = []
     for i in range(len(data.axes[1])):
      if i== len(data.axes[1])-1:
        data_name = data_name + ['Emotions']
      else:
        data_name = data_name + [str(i)]

     data_n = data_name
     data.columns = data_name
     data=normalize(data)
    #  data = LLEcomp(data, nn,mm) 
    #  data,_ = PComp(data, 20)
    
     nf = data.shape[1]-1 # número de features (componentes)
     nc = nf # número de componetes (features) a utilizar
     col = list(range(nc))
    #  print(nf)

     random.seed(w)  # 150
     rng=np.random.RandomState(j) # 155, 555

     acc_acu=0
     accuracy_acc=0
     N=int(len(data)-k)
     step=1
     i=0
     for n in range(0,N,step):
       i+=1
       wd=data
       #print('i',i)
       #print('shuffle in',data)
       ar=suffle_df(wd, 0, n, k)
       #print('shuffle out',data)
       X_train, X_test, y_train, y_test = train_test_sets(ar, 0, k, Eps, prop,nn,mm)
      #  clf1 = SVC(kernel='rbf')
      #  clf2 = SVC(kernel='poly')
      #  clf3 = GaussianNB()
      #  clf4 = RandomForestClassifier()
      #  model = VotingClassifier(estimators = [('SVCrbf', clf1), ('SVCpoly', clf2), ('gnb', clf3), ('rf', clf4)], voting ='hard')
       #X_train, X_test, y_train, y_test = train_test_sets(ar, 0, k, prop)
       #print('X_train=', X_train.shape[0])
       # --------------------------------------------------------------
       ##model = DecisionTreeClassifier(max_depth = 15)
      #  model = RandomForestClassifier()
       ##model = LinearSVC(C=2, tol = 1e-5, max_iter = 500000)
      #  model = SVC(kernel='poly') # random.seed(50), pr=0.2, rand in augmented data=117 : 0.887
      #  model = GaussianNB() # norm01: 0.873239, 0.859, 0.8450, 0.8169

       ##model = MultinomialNB(class_prior=[0.55, 0.45])  # class_prior can be used to deal with class inbalance
       ##model = LogisticRegression(C=5, tol = 1e-5, max_iter = 1000000)
       ##model = LogisticRegressionCV(cv=CV, solver='liblinear', scoring='accuracy', random_state=random.randint(1,1000), n_jobs=-1, verbose=0, max_iter=LR_max_iter) # norm01: 0.85915 
       ##model = LogisticRegressionCV(cv=CV, scoring='accuracy', random_state=random.randint(1,1000), n_jobs=-1, verbose=0, max_iter=LR_max_iter) # norm01 87.3
       model = SVC(kernel='rbf') # random.seed(50), pr=0.2, rand in augmented data=117 : 0.887

       #Fitting de Random Forest
       model.fit(X_train, y_train)

       acc = model.score(X_test, y_test)
       acc_acu += acc

       predictions = model.predict(X_test)
       cm = confusion_matrix(y_test, predictions)
       accuracy= metrics.accuracy_score(y_test, predictions)
       accuracy_acc += accuracy
     #print('w=',w,'j=',j,"average accuracy=",acc_acu/i)

     if ((w+j)%100==0):
       print('w=',w,'j=',j,"average accuracy=",acc_acu/i)
       print('Execution time: ', time.time()-start_time)


     acc_ave=acc_acu/i
     if acc_ave > acc_max:
       acc_max=acc_ave
       print('****** w=',w,'j=',j,"average accuracy=",acc_acu/i,'************')


OTH
60
5
dep_writing_Total_25_oth0.15_ith0.5.csv
w= 200 j= 200 average accuracy= 0.96875
Execution time:  11.650394201278687
****** w= 200 j= 200 average accuracy= 0.96875 ************
****** w= 200 j= 203 average accuracy= 0.9765625 ************
w= 200 j= 300 average accuracy= 0.96875
Execution time:  1127.696886062622
****** w= 200 j= 353 average accuracy= 0.984375 ************
w= 201 j= 299 average accuracy= 0.9609375
Execution time:  3356.2060146331787
w= 201 j= 399 average accuracy= 0.9765625
Execution time:  4475.653032064438
w= 202 j= 298 average accuracy= 0.9609375
Execution time:  5586.819779634476
w= 202 j= 398 average accuracy= 0.96875
Execution time:  6716.065522909164
w= 203 j= 297 average accuracy= 0.9765625
Execution time:  7829.258897781372
w= 203 j= 397 average accuracy= 0.96875
Execution time:  8958.554121732712
w= 204 j= 296 average accuracy= 0.9609375
Execution time:  10083.043713569641
w= 204 j= 396 average accuracy= 0.96875
Execution time:  11208.63200044632
w= 20